In [1]:
import collections 
from itertools import chain
import torch 
import time 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys 
import os

from torch.utils.data.dataset import Dataset
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.utils.data.dataloader import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.nn import CrossEntropyLoss

## 通用folder
from src.dataset.converter import data_loader 
from src.train_utils import set_seed, ModelSave, get_torch_device, EarlyStop, TrainParams
from src.evaluation import seqlabel_inference, seqlabel_report
from src.metric import  seq_tag_metrics, tag_cls_log
from src.seqlabel_utils import extract_entity, get_entity_bio
from src.models.bert_crf import BertCrf

## 项目folder
from models import BertCrf
from process import Schema2Label
from dataset import SeqLabelDataset
import transformers 

from transformers import BertTokenizer
transformers.logging.set_verbosity_error()
device = get_torch_device()
set_seed(1234)

No GPU available, using the CPU instead.


In [14]:
DIR = './trainsample'
schema = Schema2Label(DIR+'/duee_event_schema.json')
tp = TrainParams(
    log_steps = 50,
    save_steps = 10,
    epoch_size=20,
    max_seq_len=400,
    batch_size=20,
    loss_fn=nn.CrossEntropyLoss(),
    lr=5e-5,
    crf_lr = 5e-5 * 10,
    weight_decay=0.0,
    epsilon=1e-6,
    warmup_steps=100,
    dropout_rate=0.2,
    gradient_clip=5.0,
    early_stop_params = {
        'monitor':'f1_micro',
        'mode':'max',
        'min_delta': 0,
        'patience':3,
        'verbose':False
    },
    pretrain_model = 'bert-base-chinese',
    continue_train=False,
    label2idx = schema.event_bio_label,
    idx2label = {j:i for i,j in schema.event_bio_label.items()},
    label_size=len(schema.event_bio_label),
    schema='BIO',
    num_layers=1
)

In [8]:
tokenizer = BertTokenizer.from_pretrained(tp.pretrain_model, do_lower_case=True)
train_dataset = SeqLabelDataset(data_loader(DIR+'/train_event_bio.txt'),
                                tokenizer, tp.max_seq_len, tp.label2idx)
valid_dataset = SeqLabelDataset(data_loader(DIR+'/valid_event_bio.txt'), 
                                tokenizer, tp.max_seq_len,  tp.label2idx)
train_sampler = RandomSampler(train_dataset)
valid_sampler = SequentialSampler(valid_dataset)
train_loader = DataLoader(train_dataset, sampler=train_sampler, batch_size=tp.batch_size)
valid_loader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=tp.batch_size)

In [6]:
tp.update({'num_train_steps': int(len(train_loader)*tp.epoch_size)})

CKPT = './checkpoint/event_bio'
saver = ModelSave(CKPT, continue_train=False)
saver.init()
es = EarlyStop(**tp.early_stop_params)
global_step = 0
tb = SummaryWriter(CKPT)

model = BertCrf(tp)
model.to(device)
optimizer, scheduler = model.get_optimizer()

Downloading:   0%|          | 0.00/393M [00:00<?, ?B/s]

In [7]:
for epoch_i in range(tp['epoch_size']):
    print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10}  | {'Elapsed':^9}")
    print("-"*60)

    # Measure the elapsed time of each epoch
    t0_epoch, t0_batch = time.time(), time.time()
    total_loss, batch_loss, batch_counts = 0, 0, 0

    model.train()
    for step, batch in enumerate(train_loader):
        global_step +=1
        batch_counts +=1

        #Forward propogate
        model.zero_grad()
        feature = {k:v.to(device) for k, v in batch.items()}
        logits = model(feature)
        loss = model.compute_loss(feature, logits)
        batch_loss += loss.item()
        total_loss += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), tp.gradient_clip)
        optimizer.step()
        scheduler.step()
        # Log steps for train loss logging
        if (step % tp.log_steps == 0 and step != 0) or (step == len(train_loader) - 1):
            time_elapsed = time.time() - t0_batch
            print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^9} | {time_elapsed:^9.2f}")
            tb.add_scalar('loss/batch_train', batch_loss / batch_counts, global_step=global_step)
            batch_loss, batch_counts = 0, 0
            t0_batch = time.time()

    # On Epoch End: calcualte train & valid loss and log overall metrics
    time_elapsed = time.time() - t0_epoch
    val_metrics = seq_tag_metrics(model, valid_loader, tp.idx2label, tp.schema, device)
    avg_train_loss = total_loss / step

    print("-"*70)
    print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_metrics['val_loss']:^10.6f} | {time_elapsed:^9.2f}")
    tag_cls_log(epoch_i, val_metrics)
    print("\n")
    if es.check(val_metrics):
        break 

 Epoch  |  Batch  |  Train Loss  |  Val Loss   |  Elapsed 
------------------------------------------------------------
   1    |   50    |  119.397535  |     -     |   57.22  
   1    |   100   |  12.367022   |     -     |   54.73  
   1    |   150   |   6.609672   |     -     |   54.90  
   1    |   200   |   5.200107   |     -     |   54.84  
   1    |   250   |   3.657232   |     -     |   55.10  
   1    |   300   |   3.326165   |     -     |   55.12  
   1    |   350   |   3.045415   |     -     |   55.41  
   1    |   400   |   3.307062   |     -     |   54.56  
   1    |   450   |   2.517048   |     -     |   54.96  
   1    |   500   |   2.265942   |     -     |   54.94  
   1    |   550   |   2.178024   |     -     |   55.22  
   1    |   595   |   2.031667   |     -     |   49.19  
----------------------------------------------------------------------
   1    |    -    |  14.125015   |  2.332066  |  656.18  


 Epoch  | Macro Precision | Macro Recall | Macro F1 
------------

In [8]:
pred = seqlabel_inference(model, valid_loader, device)
valid = pd.DataFrame(valid_dataset.raw_data)
valid['pred_pos'] = [get_entity_bio(i, tp.idx2label) for i in pred]
valid['pred_entity'] = valid.apply(lambda x: extract_entity(x.text1, x.pred_pos), axis=1)

In [9]:
test_dataset = SeqLabelDataset(data_loader(DIR+'/test_event_bio.txt'), tokenizer, tp.max_seq_len,  tp.label2idx)
test_sampler = SequentialSampler(test_dataset)
test_loader = DataLoader(test_dataset, sampler=test_sampler, batch_size=tp.batch_size)
test = pd.DataFrame(test_dataset.raw_data)
pred = seqlabel_inference(model, test_loader, device)
test['pred_pos'] = [get_entity_bio(i, tp.idx2label) for i in pred]
test['pred_entity'] = test.apply(lambda x: extract_entity(x.text1, x.pred_pos), axis=1)

In [ ]:
valid.to_csv(DIR+ '/valid_event_bio_pred.csv')
test.to_csv(DIR+ '/test_event_bio_pred.csv')

###  Eval

In [24]:
from evaluation import multilabel_evaluation 
valid['true_event'] = valid['label'].map(lambda x: [i[0] for i in x])
valid['pred_event'] = valid['pred_entity'].map(lambda x: set(x.keys()))

In [56]:
multilabel_evaluation(valid['true_event'], valid['pred_event'], True)

   n_sample  n_pos  precision    recall        f1  accuracy
0      1492   1651   0.933649  0.954573  0.943995  0.893931
